In [42]:
#Problema de negócio!!!
#Popularidade de fintechs por gênero
#Quantas pessoas utilizam cartão de crédito nas compras?
#De quais regiões os usuários estão acessando (com o auxílio de bibliotecas de geolocalização de IP)

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
#Carregando os dados com pandas
dados = pd.read_csv('data.csv')
df =  pd.DataFrame(dados)

In [ ]:
#Analise exploratória

valores_ausentes = df.isnull().sum()
print(valores_ausentes)
print(df.columns)
print(df.shape)


In [ ]:
#Analise exploratória
#Não existe valores duplicados

valor_duplicado = df.duplicated()
print(valor_duplicado)

In [ ]:
#Visualização dos dados
df.head(5)

In [ ]:
# Concatenei a coluna nome e sobrenome, deixando apenas em uma coluna com rotulo Name
df['name'] = df['first_name'].str.cat(df['last_name'], sep=' ')
df = df.drop(['first_name', 'last_name'], axis=1)
df.head(3)


In [ ]:
#Reordenei as colunas
df = df[['id','name', 'email', 'gender', 'ip_address', 'Credit_card', 'Salles_Value',
       'Manufacturer']]
print(df.columns)

In [ ]:
#renomeando a coluna
df.rename(columns={'Manufacturer': 'Fintech'}, inplace=True)
df.rename(columns={'Salles_Value': 'Value'}, inplace=True)
print(df.columns)

In [ ]:
#Popularidade de fintechs por gênero
#verificando a frequencia de valores em Fintech
print(df['gender'].value_counts())



In [ ]:
#Popularidade de fintechs por gênero
#verificando a frequencia de valores em Fintech
print('--------Fintechs-----------')
print(df['Fintech'].value_counts())

In [ ]:
#Padronizando para o clustering

frequencia_fintech =  df['Fintech'].value_counts(normalize=True)
fintech_peso = frequencia_fintech.to_dict()

df['Fintech_weight'] = df['Fintech'].map(fintech_peso)
 
print(df['Fintech_weight'])

x_fintech = np.array(df['Fintech_weight'])
x = x_fintech.reshape(-1,1)

print(x.shape)

In [ ]:
#Padronizando para o clustering

#verifica quantas vezes aparece os valores
frequencia_gender =  df['gender'].value_counts(normalize=True)
gender_peso = frequencia_gender.to_dict()

df['gender_weight'] = df['gender'].map(gender_peso)

y_gender = np.array(df['gender_weight'])
y = y_gender.reshape(-1,1)

print(y.shape)

In [55]:
#pacotes para utilização do clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [56]:
#Como são Dados Categoricos, é necessário fazer a transformação para dados numéricos

In [ ]:
#usando clustering para analizar a coluna

ohe = OneHotEncoder()


#precisa transformar em um array para poder concatenar 
fintech_ohe = ohe.fit_transform(x).toarray()
gender_ohe = ohe.fit_transform(y).toarray()

data_enconded = np.concatenate((fintech_ohe, gender_ohe), axis=1)

print(data_enconded)


In [ ]:
# usando KMeans

#criando um modelo Kemeans
k = 5

kmeans = KMeans(n_clusters= k)

df['cluster'] = kmeans.fit_predict(data_enconded)

print(df.columns)

In [ ]:
#conversão da coluna Value em float, sem querer criei a coluna Valor

df['Valor'] = df['Value'].replace({'\$': '', ',': ''}, regex=True).str.strip().astype(float)

print(df['Valor'].head())

In [ ]:
print(df.columns)

df_powerBi = df[['id', 'cluster']]
df_powerBi.to_csv('dados_cluster.csv')

In [ ]:
#Calcular as porcentagens de utilização de fintechs por gênero

#usando groupby para calcular a porcentagem
agrupando_colunas =  df[['gender', 'Fintech','Valor']].groupby(['gender', 'Fintech']).sum().reset_index()

somando_valores_genero = df.groupby('gender')['Valor'].sum().reset_index()
#Somando todos os valores da coluna Valor
soma_total_coluna = df['Valor'].round(2).sum()

somando_valores_genero['porcentagem'] = (somando_valores_genero['Valor'] / soma_total_coluna) * 100

#organizando o Data Frame em ordem decrescente
somando_valores_genero = somando_valores_genero.sort_values(by='porcentagem', ascending= False)

porcentagem = df['porcentagem']
#código de porcentagem do valor

print(somando_valores_genero.head())
print('A soma total é:', soma_total_coluna)
print('A porcentagem é de:', porcentagem)
print(df['gender'].value_counts())

In [ ]:
#criação de visualização (grafico de barras ou pizza para ilustrar)

#Gráfico de barras matplotlib e seaborn

sns.set_theme(style= 'darkgrid')
plt.figure(figsize= (10,6))
bar_plot = sns.barplot(data = somando_valores_genero, x='gender', y='porcentagem', palette='viridis')
plt.xlabel("")
plt.ylabel("")
plt.title('Gênero por Fintech')


for p in bar_plot.patches:
    bar_plot.annotate(f'{p.get_height():.2f}%', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha='center', va='bottom')

plt.show()

In [ ]:
#filtrando as colunas e gravando em outro arquivo
print(df.columns)

df_Novo = df[['id', 'name','Credit_card', 'Valor','Value', 'Fintech']]

print(df_Novo)

df_Novo.to_csv('dados_analise_credit.csv', index=False)

In [ ]:
df_Novo['Valor'].round(2).describe() 


In [ ]:
df['Valor', 'gender'].describe().sum()

In [ ]:

#Analise de dados Geo
print(df.columns)

df_analise_geo = df[['id','name','gender','ip_address','Valor']]

df_analise_geo.to_csv('data_geo.csv', index= False)